# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv("../WeatherPy/output_data/cleancitydata.csv")
weather_data_df

,city,country,latitude,longitude,temp,humid,cloud,wind
0,Grindavik,IS,63.84,-22.43,48.20,93,90,2.24
1,Albany,US,42.60,-73.97,69.01,100,31,3.18
2,Busselton,AU,-33.65,115.33,63.00,48,100,8.39
3,Cape Town,ZA,-33.93,18.42,64.00,87,75,3.36
4,Port Alfred,ZA,-33.59,26.89,64.44,59,11,25.77
...,...,...,...,...,...,...,...,...
575,Novikovo,RU,51.15,37.89,72.27,75,99,6.55
576,Husavik,IS,66.04,-17.34,50.00,87,60,5.82
577,Kincardine,GB,56.07,-3.72,53.01,100,75,5.82
578,Maromitsa,RU,59.87,48.01,64.87,97,100,6.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
weather_data_df = weather_data_df.dropna()

locations = weather_data_df[["latitude", "longitude"]].astype(float)
humidity = weather_data_df["humid"].astype(float)

In [5]:
fig = gmaps.figure(center=(30, 20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(
    locations, weights=humidity, 
    max_intensity=weather_data_df["humid"].max(), 
    dissipating=False, 
    point_radius = 1)

fig.add_layer(heat_layer)
    
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
my_ideal_weather_locations_df = weather_data_df.loc[(weather_data_df["temp"] > 65) & \
                                                    (weather_data_df["temp"] < 75) & \
                                                    (weather_data_df["humid"] < 80) & \
                                                    (weather_data_df["humid"] > 50) & \
                                                    (weather_data_df["cloud"] < 20) & \
                                                    (weather_data_df["wind"] < 20)]
my_ideal_weather_locations_df

,city,country,latitude,longitude,temp,humid,cloud,wind
57,Kyshtovka,RU,56.55,76.63,67.05,54,14,7.85
64,Airai,TL,-8.93,125.41,65.61,66,0,4.21
79,Carnarvon,AU,-24.87,113.63,71.60,64,0,12.75
92,Laguna,US,38.42,-121.42,68.00,63,1,5.82
99,Geraldton,AU,-28.77,114.60,69.80,60,0,13.87
144,Jacareacanga,BR,-6.22,-57.75,67.69,60,0,1.39
223,Mossamedes,AO,-15.20,12.15,72.99,51,0,6.29
271,Balsas,BR,-7.53,-46.04,72.09,59,19,5.30
384,Vernon,CA,50.26,-119.27,66.00,72,1,3.36
406,Powell River,CA,49.85,-124.54,66.20,68,5,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = my_ideal_weather_locations_df[["latitude", "longitude", "city", "country"]]
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.rename(columns= {"latitude": "Lat", "longitude": "Lng", "city": "City", "country": "Country"})

hotel_df

C:\Users\Mama\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Lat,Lng,City,Country,Hotel Name
57,56.55,76.63,Kyshtovka,RU,
64,-8.93,125.41,Airai,TL,
79,-24.87,113.63,Carnarvon,AU,
92,38.42,-121.42,Laguna,US,
99,-28.77,114.60,Geraldton,AU,
144,-6.22,-57.75,Jacareacanga,BR,
223,-15.20,12.15,Mossamedes,AO,
271,-7.53,-46.04,Balsas,BR,
384,50.26,-119.27,Vernon,CA,
406,49.85,-124.54,Powell River,CA,


In [8]:
# assemble url and make API request
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

#    print(f"Retrieving Results for Index: {index}, hotel in {row['City Name']}")
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 50000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
        }
    
    # Pull data from API
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        hotel_df.loc[index, "Lat"] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = results[0]["geometry"]["location"]["lng"]
           
    except (KeyError, IndexError):
        pass

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Display figure
fig = gmaps.figure(center=(30, 20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(
    locations, 
    dissipating=False, 
    point_radius = 1)

fig.add_layer(heat_layer)

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))